In [35]:
%load_ext jupyternotify

import pandas as pd
import numpy as np
import time
import utils

from tqdm.auto import tqdm

pd.set_option('display.max_columns', None)

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [36]:
# pd.set_option('display.max_colwidth', None)

In [37]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
raw_all_df = pd.read_csv('../data/PitchData_15-22.csv', index_col=[0])
reg_all_df = raw_all_df[raw_all_df['game_type']=='R'].copy()
reg_all_df.reset_index(inplace=True, drop=True)

# create seperate pitch index for later merging operation
reg_all_df.reset_index(inplace=True)
reg_all_df.rename(columns={'index':'pitch_index'}, inplace=True)

reg_all_df['n_count'] = 1

/var/folders/x_/1k47n6n512j4cw0810rgvky40000gn/T/ipykernel_6838/843727565.py:1: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_all_df = pd.read_csv('../data/PitchData_15-22.csv', index_col=[0])


In [5]:
# # Visual inspection
# reg_all_df = utils.get_vid_link(reg_all_df)

# remove bad PAs
PA that changed pitcher or batter or doesn't contain events

In [6]:
pa_df = reg_all_df.groupby(['game_year', 'game_pk', 'at_bat_number', 'pitcher', 'batter']).agg({
    'events':'first',
    'des':'first',
    'pitch_index':'first'
}).reset_index()
pa_df.head()

,game_year,game_pk,at_bat_number,pitcher,batter,events,des,pitch_index
0,2015,413649,1,433587,594777,strikeout,Kole Calhoun called out on strikes.,5274688
1,2015,413649,2,433587,545361,home_run,Mike Trout homers (1) on a fly ball to center ...,5274680
2,2015,413649,3,433587,405395,strikeout,Albert Pujols strikes out on a foul tip.,5274673
3,2015,413649,4,433587,459964,strikeout,Matt Joyce strikes out swinging.,5274670
4,2015,413649,5,450308,457706,field_out,"Austin Jackson grounds out, shortstop Erick Ay...",5274668


In [7]:
# create df of duplicated pa
pa_dup = pa_df[pa_df.duplicated(subset=['game_pk', 'at_bat_number'], keep='first')]
pa_dup_list = [(x, y) for x, y in zip(pa_dup['game_pk'], pa_dup['at_bat_number'])]

drop_idx = []

for pa in pa_dup_list:
    drop_idx += reg_all_df[(reg_all_df['game_pk']==pa[0])&(reg_all_df['at_bat_number']==pa[1])].index.to_list()

# drop duplicated PA
reg_all_df.drop(index=drop_idx, inplace=True)

In [8]:
reg_all_df.shape

(5272280, 94)

In [9]:
# re-create pa_df
pa_df = reg_all_df.groupby(['game_year', 'game_pk', 'at_bat_number', 'pitcher', 'batter']).agg({
    'events':'first',
    'des':'first',
    'pitch_index':'first'
}).reset_index()
pa_df.head()

,game_year,game_pk,at_bat_number,pitcher,batter,events,des,pitch_index
0,2015,413649,1,433587,594777,strikeout,Kole Calhoun called out on strikes.,5274688
1,2015,413649,2,433587,545361,home_run,Mike Trout homers (1) on a fly ball to center ...,5274680
2,2015,413649,3,433587,405395,strikeout,Albert Pujols strikes out on a foul tip.,5274673
3,2015,413649,4,433587,459964,strikeout,Matt Joyce strikes out swinging.,5274670
4,2015,413649,5,450308,457706,field_out,"Austin Jackson grounds out, shortstop Erick Ay...",5274668


In [10]:
missing_event_df = pa_df[pa_df['events'].isna()].copy()
missing_event_df[missing_event_df['des'].isna()]

,game_year,game_pk,at_bat_number,pitcher,batter,events,des,pitch_index
8453,2015,413759,62,466948,572816,None,None,5243123
14444,2015,413840,22,282332,434658,None,None,5220248
16995,2015,413873,51,112526,592743,None,None,5210438
33041,2015,414086,46,572750,648717,None,None,5151816


In [11]:
# drop pa if des is None
missing_event_df.drop(index=missing_event_df[missing_event_df['des'].isna()].index, inplace=True)

for i, r in missing_event_df.iterrows():
    missing_event_df.loc[i, 'events_imputed'] = 'BB' if 'intentionally walks' in r['des'] else 'other'

In [12]:
missing_event_df.head()

,game_year,game_pk,at_bat_number,pitcher,batter,events,des,pitch_index,events_imputed
1856,2015,413674,68,449097,452672,None,Ryan Hanigan reaches on catcher interference b...,5267816,other
5082,2015,413716,16,477132,572041,None,"With A. J. Pollock batting, Archie Bradley pi...",5255843,other
6236,2015,413731,4,518567,435062,None,"With Howie Kendrick batting, Jimmy Rollins pic...",5251510,other
6504,2015,413734,38,594987,516416,None,Carlos Gomez picked off and caught stealing 3r...,5250549,other
8798,2015,413764,42,518774,429667,None,Ryan Howard reaches on catcher interference by...,5241841,other


In [13]:
missing_event_df.shape

(1727, 9)

In [14]:
reg_all_df = reg_all_df.merge(
    missing_event_df[['pitch_index', 'events_imputed']], on=['pitch_index'], how='left', suffixes=('', '_y')
)

In [15]:
# drop pitches if des is None
reg_all_df.drop(index=reg_all_df[reg_all_df['des'].isna()].index, inplace=True)

In [16]:
reg_all_df.loc[reg_all_df['events'].isna(), 'events'] = reg_all_df['events_imputed']
reg_all_df.reset_index(drop=True, inplace=True)

In [17]:
reg_all_df.shape

(5272267, 95)

# Remove ball 4 and strike 3 rows

In [18]:
reg_all_df['balls'].value_counts()

0    2377217
1    1570376
2     891515
3     433139
4         20
Name: balls, dtype: int64

In [19]:
reg_all_df['strikes'].value_counts()

0    2122301
1    1608040
2    1541925
3          1
Name: strikes, dtype: int64

In [20]:
b4s3_df = reg_all_df[(reg_all_df['balls']==4)|(reg_all_df['strikes']==3)][['balls', 'strikes', 'game_pk', 'at_bat_number']].drop_duplicates(subset=['game_pk', 'at_bat_number'], keep='first').copy()
b4s3_df

,balls,strikes,game_pk,at_bat_number
26157,4,2,661097,58
250201,1,3,662725,21
650016,4,2,662390,63
1085773,4,2,633530,44
1438229,4,2,630892,74
2217116,4,1,566783,67
2671331,4,2,530969,66
2997499,4,2,529872,28
3357945,4,2,491781,45
4738467,4,2,415513,4


In [21]:
# remove pa with ball 4 and strike 3

drop_idx = []
for i, r in b4s3_df.iterrows():
    drop_idx += reg_all_df[(reg_all_df['game_pk']==r[2])&(reg_all_df['at_bat_number']==r[3])].index.to_list()

In [22]:
reg_all_df[(reg_all_df['game_pk']==r[2])&(reg_all_df['at_bat_number']==r[3])].index

Int64Index([5214816, 5214817, 5214818, 5214819, 5214820, 5214821, 5214822], dtype='int64')

In [23]:
reg_clean_df = reg_all_df.drop(index=drop_idx)

In [24]:
reg_clean_df[(reg_clean_df['game_pk']==r[2])&(reg_clean_df['at_bat_number']==r[3])].index

Int64Index([], dtype='int64')

In [25]:
#pitches dropped
reg_all_df.shape[0] - reg_clean_df.shape[0]

92

# Create event_type

In [26]:
B1_list = 'single'
B2_list = 'double'
B3_list = 'triple'
HR_list = 'home_run'
FO_list = ['field_out', 'grounded_into_double_play', 'sac_bunt', 'force_out', 'field_error', 
            'sac_fly', 'double_play', 'fielders_choice', 'fielders_choice_out', 'sac_fly_double_play',
           'sac_bunt_double_play', 'triple_play']
K_list = ['strikeout', 'strikeout_double_play']
BB_list = ['walk','hit_by_pitch', 'intent_walk', 'BB']
other_list = ['caught_stealing_2b', 'caught_stealing_home','other_out', 'pickoff_3b', 'catcher_interf', 
              'caught_stealing_3b', 'wild_pitch', 'passed_ball', 'stolen_base_2b', 'pickoff_2b', 
              'pickoff_1b', 'pickoff_caught_stealing_2b', 'game_advisory', 'pickoff_caught_stealing_3b',
              'pickoff_caught_stealing_home', 'runner_double_play', 'other', 'stolen_base_home',
             'pickoff_error_2b', 'ejection', 'pickoff_error_2b', 'stolen_base_3b']

In [27]:
# check if any events are not counted
e_list = reg_clean_df['events'].unique()
[e for e in e_list if e not in FO_list + K_list + BB_list + other_list + [B1_list] + [B2_list] + [B3_list] + [HR_list]]

[nan]

In [28]:
reg_clean_df.loc[reg_clean_df['events']=='single', 'event_type'] = '1B'
reg_clean_df.loc[reg_clean_df['events']=='double', 'event_type'] = '2B'
reg_clean_df.loc[reg_clean_df['events']=='triple', 'event_type'] = '3B'
reg_clean_df.loc[reg_clean_df['events']=='home_run', 'event_type'] = 'HR'
reg_clean_df.loc[reg_clean_df['events'].isin(FO_list), 'event_type'] = 'FO'
reg_clean_df.loc[reg_clean_df['events'].isin(K_list), 'event_type'] = 'K'
reg_clean_df.loc[reg_clean_df['events'].isin(BB_list), 'event_type'] = 'BB'
reg_clean_df.loc[reg_clean_df['events'].isin(other_list), 'event_type'] = 'other'

In [ ]:
# drop un-needed columns to increase MC run speed
col_keep = [
    'pitch_index', 'game_year', 'game_pk', 'at_bat_number', 
    'home_team', 'away_team', 'pitch_type', 'game_date',
    'player_name', 'batter', 'pitcher', 'stand', 'p_throws', 
    'event_type', 'des', 'inning', 'inning_topbot',
    'balls', 'strikes', 'n_count'
]

reg_clean_lean_df = reg_clean_df[col_keep].copy()

In [41]:
# reg_clean_lean_df.to_csv('data/PitchDataCleanedLean_15-22.csv')

# Group PA

In [38]:
reg_clean_lean_df = pd.read_csv('../data/PitchDataCleanedLean_15-22.csv', index_col=[0])

In [42]:
pa_df = reg_clean_lean_df.groupby(['game_year', 'game_pk', 'at_bat_number', 'pitcher', 'batter']).agg({
    'home_team':'first',
    'away_team':'first',
    'inning_topbot':'first',
    'stand':'first',
    'p_throws':'first',
    'event_type':'first',
    'des':'first'
}).reset_index()

event_df = pd.get_dummies(pa_df['event_type'])

event_df = event_df[utils.EVENT_LIST].copy()

pa_df = pd.concat([pa_df, event_df], axis=1)

In [43]:
pa_df.head()

,game_year,game_pk,at_bat_number,pitcher,batter,home_team,away_team,inning_topbot,stand,p_throws,event_type,des,1B,2B,3B,HR,FO,BB,K,other
0,2015,413649,1,433587,594777,SEA,LAA,Top,L,R,K,Kole Calhoun called out on strikes.,0,0,0,0,0,0,1,0
1,2015,413649,2,433587,545361,SEA,LAA,Top,R,R,HR,Mike Trout homers (1) on a fly ball to center ...,0,0,0,1,0,0,0,0
2,2015,413649,3,433587,405395,SEA,LAA,Top,R,R,K,Albert Pujols strikes out on a foul tip.,0,0,0,0,0,0,1,0
3,2015,413649,4,433587,459964,SEA,LAA,Top,L,R,K,Matt Joyce strikes out swinging.,0,0,0,0,0,0,1,0
4,2015,413649,5,450308,457706,SEA,LAA,Bot,R,R,FO,"Austin Jackson grounds out, shortstop Erick Ay...",0,0,0,0,1,0,0,0


In [46]:
pa_df_1519 = pa_df[~pa_df['game_year'].isin([2022, 2021, 2020])].copy()

In [49]:
# pa_df_1519.to_csv('../data/PA_data_15-19.csv')

# pa_df.to_csv('../data/PA_data_15-22.csv')